<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E9%8A%80%E8%A1%8C%E5%88%86%E8%A1%8C%E5%9C%B0%E5%9D%80%E7%88%AC%E8%9F%B2_%E5%9C%B0%E5%9D%80%E6%89%93%E9%BB%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install fake_useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import time

import numpy as np
import pandas as pd
import random
import os


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType

import time

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [ ]:
'''
%%time
all_list = []
for i in range(1,27):
  url = 'https://www.taishinbank.com.tw/TSB/service-and-support/branch-finder/?page={}'.format(i)
  headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
  response = requests.get(url=url, headers=headers)
  soup = BeautifulSoup(response.text, 'lxml')

  title_list = soup.find_all("span", {"class": "title-detail"})
  detail_list = soup.find_all("ul", {"class": "list-detail-block"})

  for x in enumerate(title_list):
    title = x[1].text
    brn_list = []
    brn_list.append(title)
    for x in detail_list[x[0]].find_all("li"):
      brn_list.append(x.text)
    all_list.append(brn_list)
  time.sleep(3)

brn_df = pd.DataFrame(np.array(all_list),columns=['brn','addr','tel','cd','ins_box'])
brn_df["cd"] = brn_df["cd"].str.replace("分行代碼：","")
brn_df['country'] = brn_df['addr'].apply(lambda x: x[0:3])
'''

In [ ]:
"""
brn_df.to_csv('taishin_brn.csv',index = False)
!cp taishin_brn.csv /content/drive/MyDrive
"""

In [ ]:
brn_df = pd.read_csv('/content/drive/MyDrive/taishin_brn.csv')

In [ ]:
proxy_pool = pd.read_csv('/content/drive/MyDrive/proxy_ip_pool.csv')

In [ ]:
#ip_pool_args = ['--proxy-server=http://{ip}:{port}'.format(ip=x[0], port=x[1]) for x in list(zip(proxy_pool['IP'], proxy_pool['Port']))]

In [ ]:
#ua = UserAgent()
#user_agent = ua.random
#options.add_argument("user-agent={}".format(user_agent))

In [ ]:
co = webdriver.ChromeOptions()
co.add_argument("--headless")
co.add_argument('--no-sandbox')
co.add_argument('--headless')
co.add_argument('--disable-dev-shm-usage')
co.add_argument('--disable-browser-side-navigation')
co.add_experimental_option('excludeSwitches', ['enable-automation']) 


In [ ]:
def get_coordinate():
    addr=addr_list[-1]
    prxy_drv.implicitly_wait(10) # seconds
    prxy_drv.get("http://www.map.com.tw/")
    
    try:
      element = WebDriverWait(prxy_drv, 10).until(
          EC.presence_of_element_located((By.ID, "searchWord"))
            )
    
    except Exception as e:
      print(e)
      prxy_drv.quit()

    search = prxy_drv.find_element_by_id("searchWord")
    search.clear()
    print('成功進入')

    ## 輸入地址
    search.send_keys(addr)
    time.sleep(float(np.random.uniform(low=0.8, high=1, size=(1))))
    prxy_drv.find_element_by_xpath("/html/body/form/div[10]/div[2]/img[2]").click() 
    time.sleep(float(np.random.uniform(low=3, high=7, size=(1))))

    ## 更新點
    ## 這網頁有多個iframe, 要switch到資訊所在iframe
    iframe = prxy_drv.find_elements_by_class_name("winfoIframe")[0]
    prxy_drv.switch_to.frame(iframe)
    coor_btn = prxy_drv.find_element_by_xpath("/html/body/form/div[4]/table/tbody/tr[3]/td/table/tbody/tr/td[2]")
    coor_btn.click()

    ## 座標元素
    coor = prxy_drv.find_element_by_xpath("/html/body/form/div[5]/table/tbody/tr[2]/td")
    coor = coor.text.strip().split(" ")
    lat = coor[-1].split("：")[-1]
    log = coor[0].split("：")[-1]
    print('地址',addr, '點位爬取成功')
    res = (addr, lat, log)
    addr_list.pop()
    return res

In [ ]:
def get_proxies(co=co):
    driver = webdriver.Chrome(executable_path='chromedriver',options=co)
    driver.get("https://free-proxy-list.net/")

    PROXIES = []
    proxies = driver.find_elements_by_css_selector("tr[role='row']")
    for p in proxies:
        result = p.text.split(" ")

        if result[-1] == "yes":
            PROXIES.append(result[0]+":"+result[1])

    driver.close()
    return PROXIES

def proxy_driver(PROXIES, co=co):
    prox = Proxy()

    if len(PROXIES) < 1:
        print("--- Proxies used up (%s)" % len(PROXIES))
        PROXIES = get_proxies()
        
    pxy = PROXIES[-1]

    prox.proxy_type = ProxyType.MANUAL
    prox.http_proxy = pxy
    #prox.socks_proxy = pxy
    #prox.ssl_proxy = pxy

    capabilities = webdriver.DesiredCapabilities.CHROME
    prox.add_to_capabilities(capabilities)

    driver = webdriver.Chrome(options=co, desired_capabilities=capabilities)

    return driver

In [ ]:
#ALL_PROXIES = get_proxies()

In [ ]:
ALL_PROXIES = ['{ip}:{port}'.format(ip=x[0], port=x[1]) for x in list(zip(proxy_pool['IP'], proxy_pool['Port']))]

In [ ]:
##https://zhuanlan.zhihu.com/p/61536685

In [ ]:
prxy_drv = proxy_driver(ALL_PROXIES)
running = True
addr_list = list(brn_df['addr'])

In [ ]:
xy_list = []

In [ ]:
while running:
    try:
        addr=addr_list[-1]
        prxy_drv.implicitly_wait(10) # seconds
        prxy_drv.get("http://www.map.com.tw/")
        
        try:
          element = WebDriverWait(prxy_drv, 10).until(
              EC.presence_of_element_located((By.ID, "searchWord"))
                )
        
        except Exception as e:
          print(e)
          prxy_drv.quit()

        search = prxy_drv.find_element_by_id("searchWord")
        search.clear()
        print('成功進入')

        ## 輸入地址
        search.send_keys(addr)
        time.sleep(float(np.random.uniform(low=0.8, high=1, size=(1))))
        prxy_drv.find_element_by_xpath("/html/body/form/div[10]/div[2]/img[2]").click() 
        time.sleep(float(np.random.uniform(low=3, high=7, size=(1))))

        ## 更新點
        ## 這網頁有多個iframe, 要switch到資訊所在iframe
        iframe = prxy_drv.find_elements_by_class_name("winfoIframe")[0]
        prxy_drv.switch_to.frame(iframe)
        coor_btn = prxy_drv.find_element_by_xpath("/html/body/form/div[4]/table/tbody/tr[3]/td/table/tbody/tr/td[2]")
        coor_btn.click()

        ## 座標元素
        coor = prxy_drv.find_element_by_xpath("/html/body/form/div[5]/table/tbody/tr[2]/td")
        coor = coor.text.strip().split(" ")
        lat = coor[-1].split("：")[-1]
        log = coor[0].split("：")[-1]
        print('地址',addr, '點位爬取成功')
        res = (addr, lat, log)
        addr_list.pop()


        xy_list.append(res)
        # if statement to terminate loop if code working properly
        # you need to modify condition_met
        if len(addr_list)==0:
            running = False
        
        # you 
    except Exception as e:
        print(e)
        new = ALL_PROXIES.pop()
        prxy_drv.quit()
        time.sleep(2)
        os.system("taskkill /im chromedriver.exe")
        # reassign driver if fail to switch proxy
        prxy_drv = proxy_driver(ALL_PROXIES)
        print("--- Switched proxy to: %s" % new)
        time.sleep(1)

In [ ]:
xy_df = pd.DataFrame(xy_list,columns = ['addr','lat','lon'])

In [ ]:
xy_df.to_csv('brn_xy_df.csv',index = False)

In [ ]:
#!cp brn_xy_df.csv /content/drive/MyDrive